1. Обновление списка бумаг и дозагрузка данных, включая ставку ЦБ РФ, курс Доллар/Рубль, цены различных марок нефти, индекс Мосбиржи

In [ ]:
!python -m _tools.update_universe \
    --min-days 320

In [ ]:
import pandas as pd
from pathlib import Path
from datetime import datetime
from dateutil.relativedelta import relativedelta
from finam_export import Exporter, Market, Timeframe

# --- НАСТРОЙКИ ---
TICKER_TO_DOWNLOAD = "SBER"
YEARS_OF_HISTORY = 20
OUTPUT_DIR = Path("1_data")
# -----------------

# Гарантируем, что папка для данных существует
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
output_path = OUTPUT_DIR / f"{TICKER_TO_DOWNLOAD}_D1_MOEX.csv"

print(f"--- 🚀 Запуск надежной загрузки для тикера: {TICKER_TO_DOWNLOAD} через finam-export ---")

try:
    # --- 1. Определяем даты ---
    end_date = datetime.now().date()
    start_date = (datetime.now() - relativedelta(years=YEARS_OF_HISTORY)).date()
    
    print(f"Запрашиваемый период: с {start_date} по {end_date}.")

    # --- 2. Загрузка данных с помощью finam-export ---
    exporter = Exporter()
    # Ищем SBER на рынке акций MOEX
    sber = exporter.lookup(name=TICKER_TO_DOWNLOAD, market=Market.STOCKS)
    
    if not sber.empty:
        # Получаем ID тикера для запроса
        sber_id = sber.index[0]
        data = exporter.download(
            sber_id,
            market=Market.STOCKS,
            start_date=start_date,
            end_date=end_date,
            timeframe=Timeframe.DAILY
        )
    else:
        raise ValueError(f"Тикер {TICKER_TO_DOWNLOAD} не найден в справочнике Finam.")

    # --- 3. Форматирование и сохранение ---
    if not data.empty:
        # Переименовываем колонки в ваш формат
        # Finam использует <DATE>, <OPEN>, <HIGH>, <LOW>, <CLOSE>, <VOL>
        rename_map = {
            '<DATE>': 'datetime',
            '<OPEN>': 'open',
            '<HIGH>': 'high',
            '<LOW>': 'low',
            '<CLOSE>': 'close',
            '<VOL>': 'volume'
        }
        data.rename(columns=rename_map, inplace=True)
        
        # Преобразуем дату
        data['datetime'] = pd.to_datetime(data['datetime'], format='%Y%m%d')
        
        # Оставляем только нужные колонки
        final_df = data[list(rename_map.values())].copy()
        
        # Сортируем на всякий случай
        final_df.sort_values('datetime', inplace=True)
        
        # Сохраняем файл, полностью перезаписывая старый
        final_df.to_csv(output_path, index=False)

        print(f"\n--- 📊 Итоги загрузки ---")
        print(f"✅ Успешно сохранено {len(final_df)} строк.")
        print(f"  - Первая дата в файле: {final_df['datetime'].min().date()}")
        print(f"  - Последняя дата в файле: {final_df['datetime'].max().date()}")
        print(f"Файл сохранен в: {output_path}")

    else:
        print("⚠️ finam-export не вернул данные.")

except Exception as e:
    print(f"\n❌ Произошла ошибка при загрузке через finam-export: {e}")

--- 🚀 Запуск надежной загрузки для тикера: SBER через CLI finam-export ---
Запрашиваемый период: с 2005-10-16 по 2025-10-16.

Выполняю команду:
/home/restorator/trader_test/.venv-tf/bin/python -m finam_export SBER 1_data/SBER_D1_MOEX.csv --start-date 20051016 --end-date 20251016 --market shares --timeframe daily

--- ❌ Ошибка выполнения finam-export ---
/home/restorator/trader_test/.venv-tf/bin/python: No module named finam_export


❌ Произошла непредвиденная ошибка: CLI finam-export завершился с ошибкой.


In [19]:
import sys
!{sys.executable} -m pip install finam-export

In [15]:
!python -m _tools.download_all_data
!python -m _tools.analyze_data_coverage


--- Начинаю полную загрузку данных за 20 лет ---

[1/7] Загрузка котировок по акциям...
Загрузка акций: 100%|█████████████████████████| 243/243 [05:09<00:00,  1.27s/it]
✅ Загрузка акций завершена.
   - Новых файлов создано: 243
   - Файлов обновлено: 0
   - Актуальных файлов (без изменений): 0
   - Ошибок: 0

[2/7] Загрузка индексов МосБиржи (IMOEX/MOEXBC)...
Загрузка истории индекса IMOEX с 2005-10-15...
✅ Полная история индекса IMOEX (5010 строк) сохранена в 1_data/IMOEX.csv

[3/7] Загрузка ключевой ставки ЦБ РФ...
Загрузка полной истории ключевой ставки со страницы: https://www.cbr.ru/hd_base/keyrate/...
✅ Полная история ключевой ставки успешно сохранена в /home/restorator/trader_test/1_data/CBR_KEY_RATE.csv

[4/7] Загрузка истории официального курса USD/RUB от ЦБ РФ...
Загрузка истории USD/RUB с сайта ЦБ РФ (15/10/2005 - 15/10/2025)...
✅ Полная история USD/RUB (7306 строк) сохранена в /home/restorator/trader_test/1_data/USDRUB.csv

[5/7] Загрузка истории нефти Brent (BZ=F)...
Загр

apply_splits теперь находит все сплиты, но допускает ошибки при применении масштабирования, например он пропусти 3 дня для 

In [ ]:
!python -m _tools.apply_splits \
    --data-dir 1_data \
    --workers 8

2. Очистка бумаг, обрезая по 2 дня около длинных разрывов, с минимальным сегментом ≥5 дней:

In [20]:
!python -m _tools.clean_timeseries \
    --data-dir 1_data \
    --gap-ok 3 \
    --gap-buffer 2 \
    --min-seg-len 5 \
    --inplace \
    --remove-outliers \
    --outlier-abs-threshold 40.0 \
    --outlier-sma-window 20 \
    --outlier-sma-multiplier 9.0

Cleaning OHLCV: 100%|█████████████████████████| 249/249 [00:04<00:00, 61.72it/s]

[OK] Очистка завершена.


3. Проверка списка признаков и генерация таблиц с ними на основании загруженных данных

In [ ]:
!python -m _core.feature_generator

4. Оценка матожидания отклонения цены через выбранное количество дней на исторических данных, разметка данных с учетом отклонений

5. Ранжированмие признаков, корреляционная фильтрация, отсеивание списка N лучших

In [ ]:
!python -m _tools.generate_features --workers 7

In [ ]:
!python -m _tools.analyze_volatility_for_labeling \
    --horizon 10

In [8]:
!python -m _tools.labeler \
    --horizon 10 \
    --tp 7.0 \
    --sl 4.5 \
    --workers 7

--- Шаг 2.5: Разметка данных методом 'Тройной преграды' ---
Параметры: Горизонт=10 дней, TakeProfit=7.0%, StopLoss=4.5%
Найдено 243 файлов для обработки. Запускаю в 7 потоков...
Разметка файлов: 100%|████████████████████████| 243/243 [00:09<00:00, 25.30it/s]

✅ Процесс разметки завершен.
🎉 Все файлы обработаны без ошибок.
Обработанные файлы сохранены в: /home/restorator/trader_test/2_results/features_processed


In [1]:
MODEL_NAME="universal_60_10_v1_corr_threshold_100"
DATA_SOURCE_MODEL="universal_60_10_v1_corr_threshold_100"
MODEL_NAME_PREFIX="universal_60_10_v1"
UNIVERSE_FILE="universe.csv"

In [8]:
!python -m _tools.feature_selector \
    --model_name "{MODEL_NAME}" \
    --top_n 160 \
    --corr_threshold 1.00 \
    --repeats 3 \
    --n_splits 5 \
    --use_gpu

Загрузка файлов с признаками из: /home/restorator/trader_test/2_results/features_processed
Заполнение пропусков: 100%|█████████████████| 149/149 [00:00<00:00, 2230.18it/s]

--- 🚀 Запуск отбора признаков по метрике: 'ACCURACY' ---

--- Повтор [1/3] ---
  Фолд [1/5]...
  Фолд [2/5]...
  Фолд [3/5]...
  Фолд [4/5]...
  Фолд [5/5]...

--- Повтор [2/3] ---
  Фолд [1/5]...
  Фолд [2/5]...
  Фолд [3/5]...
  Фолд [4/5]...
  Фолд [5/5]...

--- Повтор [3/3] ---
  Фолд [1/5]...
  Фолд [2/5]...
  Фолд [3/5]...
  Фолд [4/5]...
  Фолд [5/5]...

--- Рейтинг признаков по ВЛИЯНИЮ НА ТОЧНОСТЬ (положительные = полезные) ---
Порог корреляции >= 1.0, фильтрация пропускается.

--- Итоговый рейтинг (топ-20) ---
ICS_26                   0.047060
imoex_volatility_20d     0.031529
sp500_close              0.023719
imoex_close              0.021791
usdrub_close             0.015392
usdrub_volatility_20d    0.012368
VWAP_D                   0.011251
spread_hl                0.008010
index                    0.007

6. Разделение данных train dataset и test dataset. В проверочный набор данных входят последние года всех бумаг

In [9]:
!python -m _tools.assemble_final_data \
    --model_name "{MODEL_NAME}" \
    --source_dir_name 'features_processed' \
    --workers 7
    #--source_dir_name 'features_combined/{MODEL_NAME}' \

--- ⚙️ Запуск сборки финального датасета ---
Источник файлов с признаками: /home/restorator/trader_test/2_results/features_processed
Загружено 149 отобранных признаков.
Загрузка и объединение 243 файлов в 7 потоков...
100%|████████████████████████████████████████| 243/243 [00:01<00:00, 158.84it/s]
✅ Сборка завершена. В итоговой таблице 152 колонок (149 признаков).
Точка разделения train/test: 2024-10-03
Иммутация NaN завершена. NaN в train: 117315 → 0; NaN в test: 11520 → 0

✅ Обучающий набор сохранен в: /home/restorator/trader_test/2_results/final_for_model/universal_60_10_v1_corr_threshold_100/train_final.csv
✅ Тестовый набор сохранен в: /home/restorator/trader_test/2_results/final_for_model/universal_60_10_v1_corr_threshold_100/test_final.csv


7. Обучение скелера для приведения данных к одному масштабу. Скейлер обучается только на train dataset

In [10]:
!python -m _tools.fit_scaler \
    --model_name "{MODEL_NAME}"
!python -m _tools.apply_scaler \
    --model_name "{MODEL_NAME}"

————————————————————————————————————————————————————————————
🧰 Обучение скейлера (Imputer + RobustScaler)
Модель: universal_60_10_v1_corr_threshold_100
Загрузка тренировочных данных из: /home/restorator/trader_test/2_results/final_for_model/universal_60_10_v1_corr_threshold_100/train_final.csv
————————————————————————————————————————————————————————————
Признаков: 149, обучающих сэмплов: 218232
Обучение Pipeline (Imputer + RobustScaler)...
✅ Pipeline успешно обучен.
✅ Абсолютно 'безопасный' Pipeline сохранен в: /home/restorator/trader_test/2_results/scalers/universal_60_10_v1_corr_threshold_100_scaler.pkl
🗂  Порядок признаков сохранен в: /home/restorator/trader_test/2_results/scalers/universal_60_10_v1_corr_threshold_100_features.json
Готово.
---  Applying Scaler for model: universal_60_10_v1_corr_threshold_100 ---
Loading artifacts and data...
Applying scaler transformation...
Saving scaled data...
✅ Scaled data saved to: /home/restorator/trader_test/2_results/features_scaled/universa

8. Конвертация данных в tfrecord формат данных для TensorFlow

In [11]:
!python -m _tools.convert_to_tfrecord \
    --model_name "{MODEL_NAME}" \
    --lookback 60

I0000 00:00:1760475656.575435 1132461 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1760475657.917199 1132461 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
--- Converting scaled data to TFRecord for model: universal_60_10_v1_corr_threshold_100 ---
Loading scaled data...
Writing test.tfrecord: 100%|██████████████████| 242/242 [00:16<00:00, 15.01it/s]
Calculating class weights and saving metadata...
✅ TFRecord files and metadata saved to: /home/restorator/trader_test/2_results/tfrecord_universal_60_10_v1_corr_threshold_100


9. Обучение модели

In [1]:
MODEL_NAME="universal_60_10_v1_corr_threshold_100"
DATA_SOURCE_MODEL="universal_60_10_v1_corr_threshold_100"
MODEL_NAME_PREFIX="universal_60_10_v1"
UNIVERSE_FILE="universe.csv"

In [ ]:
!python -m _tools.hyperparameter_tuner \
    --model_name "{MODEL_NAME}" \
    --n_trials 600 \
    --batch_size 4098

I0000 00:00:1760558637.028696    3704 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1760558639.168767    3704 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
[I 2025-10-15 23:04:03,791] A new study created in memory with name: universal_60_10_v1_corr_threshold_100


In [ ]:
!python -m _tools.train_universal_lstm \
    --model_name "{MODEL_NAME}" \
    --data_model_name "{DATA_SOURCE_MODEL}" \
    --params_from_tuner \
    --params_file "best_params_{MODEL_NAME}.json" \
    --runs 50 \
    --batch_size 4096 \
    --verbose 1

I0000 00:00:1760545698.563344  790164 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1760545700.309550  790164 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Использую данные от модели: 'universal_60_10_v1_corr_threshold_75'
Обученная модель будет сохранена как: 'universal_60_10_v1_corr_threshold_75'
✅ Метаданные загружены.

---  режим '--params_from_tuner' ---
Использую указанный файл с параметрами: /home/restorator/trader_test/2_results/best_params_universal_60_10_v1_corr_threshold_75_3.json
✅ Лучшие параметры успешно загружены.
{
  "conv_filters": 24,
  "kernel_size": 3,
  "gru_units_1"

In [ ]:
!python -m _tools.train_universal_lstm \
    --model_name "{MODEL_NAME}" \
    --data_model_name "{DATA_SOURCE_MODEL}" \
    --use_gpu \
    --batch_size 4096 \
    --lr 1e-2 \
    --l2_reg 2.2e-8 \
    --verbose 1 \
    --leaky_relu_negative_slope 0.062 \
    --runs 30

10. Сравнительный анализ крутизны ланшафта потерь в окресности решения (среднего значения модели на train)

In [17]:
MODEL_NAME = "universal_60_10_v1_corr_threshold_90"  # Имя вашей лучшей модели
TICKER = "YKEN"  # Тикер для теста

!python -m _tools.backtester \
    --model_name "{MODEL_NAME}" \
    --ticker "{TICKER}"

I0000 00:00:1760555941.830407  877262 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1760555943.056353  877262 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
--- 📈 Запуск бектеста для модели 'universal_60_10_v1_corr_threshold_90' на тикере 'YKEN' ---
Загрузка артефактов...
✅ Артефакты загружены. Lookback: 60, Признаков: 109
Загрузка данных для тикера из /home/restorator/trader_test/2_results/features_processed/YKEN.csv...
Подготовка 184 строк данных для модели...
Получение предсказаний модели...
Создание графика...
✅ График бектеста сохранен в: /home/restorator/trader_test/2_results/report

In [2]:
!python -m _tools.compare_model_sharpness \
    --model_prefix "{MODEL_NAME_PREFIX}" \
    --batch_size 16384 \
    --n_probes 100 \
    --noise_scale 1e-0

I0000 00:00:1760602169.495873  144405 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1760602170.883758  144405 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
--- Сравнительный детерминированный анализ резкости для моделей с префиксом: 'universal_60_10_v1' ---
Найдено 4 моделей для сравнения.
Используются данные, соответствующие каждой модели.
Анализ моделей: 100%|█████████████████████████████| 4/4 [05:10<00:00, 77.61s/it]

--- 📊 Итоговая таблица сравнения (детерминированная) ---
                                       min_loss  accuracy  sharpness
model_name                                 

11. Расчет ландшафта потерь (loss) модели и построение тепловых карт двумерных проекций модели

In [10]:
!python -m _tools.calculate_landscape \
    --model_name "{MODEL_NAME}" \
    --data_model_name "{DATA_SOURCE_MODEL}" \
    --model_filename model.keras \
    --scale 1 \
    --points 14 \
    --batch_size 16384 \
    --num_images 1

I0000 00:00:1760553515.310887  857493 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1760553516.493180  857493 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
--- Расчет ландшафта потерь для модели: model.keras ---
--- Используются данные от: universal_60_10_v1_corr_threshold_75 ---
❌ Ошибка при загрузке артефактов: STACK_GLOBAL requires str


In [ ]:
!python -m _tools.plot_landscape_from_data \
    --model_name "universal_60_10_v4_lrelu_alpha_4" \
    --model_filename model.keras \
    --vmax 1 \
    --plot_type 2d

I0000 00:00:1758742230.210234  787253 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1758742231.385963  787253 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
--- ОТРИСОВКА (2d) ландшафта для модели 'universal_60_10_v1_60_features_v2' ---
Найдено 3 файлов с данными. Начинаю отрисовку...

✅ Отрисовка завершена. 3 изображений сохранены в: /home/restorator/trader_test/2_results/reports/landscape_plots_universal_60_10_v1_60_features_v2
